# VMware Documentation Analysis Experiment

이 노트북은 단일 쿼리에 대한 모델의 분석 결과를 실행하고 시각화합니다.

In [3]:
import subprocess
import sys
from IPython.display import display, Markdown

def run_experiment():
    print("--- Experiment Execution Started ---")
    
    # Execute the main script
    result = subprocess.run(
        ["uv", "run", "python", "-m", "src.main"],
        capture_output=True,
        text=True,
        encoding="utf-8"
    )
    
    if result.returncode != 0:
        print("!! Error occurred in execution !!")
        print(result.stderr)
        return
    
    full_output = result.stdout
    
    # 단일 답변 추출 로직
    delimiter = "[Mistral-Small-2506 Answer]"
    if delimiter in full_output:
        parts = full_output.split(delimiter)
        logs = parts[0]
        answer = parts[1].replace("=", "").strip()
        
        print("### Processing Logs")
        print(logs.strip())
        
        display(Markdown("## 🎯 Analysis Result"))
        display(Markdown(answer))
    else:
        print("Error: No answer found in output.")
        print(full_output)

run_experiment()

--- Experiment Execution Started ---
Error: No answer found in output.

==================== Starting Experiment: EU AI Act Analysis ====================
--- [API Mode Active: mistral-small-latest] ---
Model Persona: Professional Technical Documentation Analyst
Adaptive Parameter Tuning: SBERT-based Semantic Analysis Enabled
Total processed chunks: 99
[Indexer] Indexed 2 chunks containing factual dates and versions.

[Running Query] How many articles are there in the AI Act?
[Orchestrator] Context Assembled: 56 fragments (224660 chars).
[AI Optimizer] Calculated Temperature: 0.3

------------------------------
[Result for: EU AI Act Analysis]
The AI Act contains 89 articles.
------------------------------
✅ Result logged to CSV.

[Running Query] Summarize the requirements for high-risk AI systems.
[Orchestrator] Context Assembled: 57 fragments (228960 chars).
[AI Optimizer] Calculated Temperature: 0.28

------------------------------
[Result for: EU AI Act Analysis]
The requirements fo